# Wikidata items

In [1]:
!pip install sparqlwrapper

import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import re


## Query Wikidata function

The function takes a SPARQL query string as its argument. It will run the SPARQL query and return the result as a data frame

In [2]:
def query_wikidata(query):
    endpoint_url = "https://query.wikidata.org/sparql"

    def get_results(endpoint_url, query):
        user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
        # TODO adjust user agent; see https://w.wiki/CX6
        sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        return sparql.query().convert()


    results = get_results(endpoint_url, query)

    raw = pd.json_normalize(results["results"]["bindings"])

    df = raw.filter(regex="\.value$")
    df = df.rename(columns=lambda x: re.sub('\.value$','',x))
    
    # order columns so that they are always in the same order
    cols = ['item', 'itemLabel', 'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 'enc_au_sc_id',
            'yob', 'yod', 'wyb', 'wye']
    df = df[cols]
    
    return df

## SPARQL queries

Because wildcard search against item label in Wikidata are very slow and generally time out, we run a number of searches on the presence of relavant identifiers.

SPARQL queries courtesy of Mathias Dillen, Botanic Garden Meise: https://github.com/matdillen/STSM-wikidata-people/blob/master/collectormatching.Rmd. I have changed them slightly by removing the identifier the presence of which is queried on from the SELECT clause and adding the Harvard Index of Botanists ID (P6264), IPNI ID (P586) and IPNI Standard Form (P428). This way all queries return the same terms and no data will be lost when duplicates are removed. The added terms will be valuable for verifying matches later on.

### Entomologists of the world (P5370)

In [3]:
queries = {}

queries['ento'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P5370 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid . }
  OPTIONAL { ?item wdt:P214 ?viaf . }
  OPTIONAL { ?item wdt:P213 ?isni . }
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### BHL creators (P4081)

In [4]:
queries['bhl_creator'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P4081 ?id .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid . }
  OPTIONAL { ?item wdt:P214 ?viaf . }
  OPTIONAL { ?item wdt:P213 ?isni . }
  OPTIONAL { ?item wdt:P6264 ?harv }
  OPTIONAL { ?item wdt:P586 ?ipni }
  OPTIONAL { ?item wdt:P428 ?abbr }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### Harvard Index of Botanists ID (P6264)

In [5]:
queries['harvard_index'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P6264 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid . }
  OPTIONAL { ?item wdt:P214 ?viaf . }
  OPTIONAL { ?item wdt:P213 ?isni . }
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### Zoobank ID (P2006)

In [6]:
queries['zoobank_id'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P2006 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid .}
  OPTIONAL { ?item wdt:P214 ?viaf .}
  OPTIONAL { ?item wdt:P213 ?isni .}
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### IPNI ID (P586)

In [7]:
queries['ipni_id'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye ?harv ?ipni ?abbr  ?bloodhound_id ?enc_au_sc_id
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P586 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid .}
  OPTIONAL { ?item wdt:P214 ?viaf .}
  OPTIONAL { ?item wdt:P213 ?isni .}
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### Wikispecies article

In [8]:
queries['wikispecies'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?id schema:about ?item ;
    schema:isPartOf <https://species.wikimedia.org/> .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }
  OPTIONAL { ?item wdt:P496 ?orcid . }
  OPTIONAL { ?item wdt:P214 ?viaf . }
  OPTIONAL { ?item wdt:P213 ?isni . }
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### Bloodhound Tracker ID (P6944)

In [9]:
queries['bloodhound_id'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P6944 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid .}
  OPTIONAL { ?item wdt:P214 ?viaf .}
  OPTIONAL { ?item wdt:P213 ?isni .}
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### Encyclopedia of Australian Science ID (P4228)

In [10]:
queries['encyclopedia_australian_science_id'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P4228 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid .}
  OPTIONAL { ?item wdt:P214 ?viaf .}
  OPTIONAL { ?item wdt:P213 ?isni .}
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

## Create the data frame

Run all the SPARQL requests, concatenate the results and drop duplicates

In [11]:
# Run the queries and create a list of data frames
frames = []
for key, query in queries.items():
    dfi = query_wikidata(query)
    print(key + ': ' + str(len(dfi.index)) + ' records')
    frames.append(dfi)

# Concatenate the dataframes from each SPARQL request
df = pd.concat(frames)

# Drop duplicates
df = df.drop_duplicates(subset=['item'])

df.head()

ento: 10662 records
bhl_creator: 24323 records
harvard_index: 23870 records
zoobank_id: 15210 records
ipni_id: 54698 records
wikispecies: 46464 records
bloodhound_id: 1632 records
encyclopedia_australian_science_id: 2798 records


,item,itemLabel,orcid,viaf,isni,harv,ipni,abbr,bloodhound_id,enc_au_sc_id,yob,yod,wyb,wye
0,http://www.wikidata.org/entity/Q67329,Carl Gustav Jablonsky,NaN,24944037,0000 0000 5526 3001,NaN,NaN,NaN,NaN,NaN,1756,1787,NaN,NaN
1,http://www.wikidata.org/entity/Q68738,Hermann Müller,NaN,2532803,0000 0001 0837 1728,36129,6735-1,H.Müll.,NaN,NaN,1829,1883,NaN,NaN
2,http://www.wikidata.org/entity/Q66902,Hermann Lebert,NaN,27833384,0000 0001 2100 1924,67704,22162-1,Lebert,NaN,NaN,1813,1878,NaN,NaN
3,http://www.wikidata.org/entity/Q69552,Carl Julius Bernhard Börner,NaN,118457204,0000 0001 1085 9124,1763,12350-1,Börner,NaN,NaN,1880,1953,NaN,NaN
4,http://www.wikidata.org/entity/Q66379,Ludwig Bechstein,NaN,36914329,0000 0001 2278 3438,NaN,NaN,NaN,NaN,NaN,1801,1860,NaN,NaN


Add surname, initials and canonical string (`surname` + ', ' + `initials`) columns 

In [16]:
surname = []
initials = []
canonical = []

for i, item in df.iterrows():
    words = re.split('[ .]', item['itemLabel'])
    words = [string for string in words if string != ""]
    surname.append(words[-1])
    initials.append(".".join([x[0] for x in words[0:-1]]) + '.')
    canonical.append(words[-1] + ', ' + ".".join([x[0] for x in words[0:-1] if len(x) > 0]) + '.')
    
df['surname'] = surname
df['initials'] = initials
df['canonical_string'] = canonical
    
df.head()

,item,itemLabel,orcid,viaf,isni,harv,ipni,abbr,bloodhound_id,enc_au_sc_id,yob,yod,wyb,wye,surname,initials,canonical_string
0,http://www.wikidata.org/entity/Q67329,Carl Gustav Jablonsky,NaN,24944037,0000 0000 5526 3001,NaN,NaN,NaN,NaN,NaN,1756,1787,NaN,NaN,Jablonsky,C.G.,"Jablonsky, C.G."
1,http://www.wikidata.org/entity/Q68738,Hermann Müller,NaN,2532803,0000 0001 0837 1728,36129,6735-1,H.Müll.,NaN,NaN,1829,1883,NaN,NaN,Müller,H.,"Müller, H."
2,http://www.wikidata.org/entity/Q66902,Hermann Lebert,NaN,27833384,0000 0001 2100 1924,67704,22162-1,Lebert,NaN,NaN,1813,1878,NaN,NaN,Lebert,H.,"Lebert, H."
3,http://www.wikidata.org/entity/Q69552,Carl Julius Bernhard Börner,NaN,118457204,0000 0001 1085 9124,1763,12350-1,Börner,NaN,NaN,1880,1953,NaN,NaN,Börner,C.J.B.,"Börner, C.J.B."
4,http://www.wikidata.org/entity/Q66379,Ludwig Bechstein,NaN,36914329,0000 0001 2278 3438,NaN,NaN,NaN,NaN,NaN,1801,1860,NaN,NaN,Bechstein,L.,"Bechstein, L."


In [18]:
df = df[['item', 'itemLabel', 
        'surname', 'initials', 'canonical_string', 
        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 'enc_au_sc_id',
        'yob', 'yod', 'wyb', 'wye']]
df.reset_index(drop=True, inplace=True)
df.head()

,item,itemLabel,surname,initials,canonical_string,orcid,viaf,isni,harv,ipni,abbr,bloodhound_id,enc_au_sc_id,yob,yod,wyb,wye
0,http://www.wikidata.org/entity/Q67329,Carl Gustav Jablonsky,Jablonsky,C.G.,"Jablonsky, C.G.",NaN,24944037,0000 0000 5526 3001,NaN,NaN,NaN,NaN,NaN,1756,1787,NaN,NaN
1,http://www.wikidata.org/entity/Q68738,Hermann Müller,Müller,H.,"Müller, H.",NaN,2532803,0000 0001 0837 1728,36129,6735-1,H.Müll.,NaN,NaN,1829,1883,NaN,NaN
2,http://www.wikidata.org/entity/Q66902,Hermann Lebert,Lebert,H.,"Lebert, H.",NaN,27833384,0000 0001 2100 1924,67704,22162-1,Lebert,NaN,NaN,1813,1878,NaN,NaN
3,http://www.wikidata.org/entity/Q69552,Carl Julius Bernhard Börner,Börner,C.J.B.,"Börner, C.J.B.",NaN,118457204,0000 0001 1085 9124,1763,12350-1,Börner,NaN,NaN,1880,1953,NaN,NaN
4,http://www.wikidata.org/entity/Q66379,Ludwig Bechstein,Bechstein,L.,"Bechstein, L.",NaN,36914329,0000 0001 2278 3438,NaN,NaN,NaN,NaN,NaN,1801,1860,NaN,NaN


In [19]:
df.to_csv('data/wikidata_persons.csv')